In [ ]:
from pandas.io.json import json_normalize
import pandas as pd
import json
from datetime import timedelta
from datetime import datetime, timedelta
import pytz

# TODO: You need to change the file path and bucket names
thinkpad_data = json.load(open('/content/drive/My Drive/Backups/ActivityWatch-ThinkPadT420-20201019.json'))
thinkpad_apps = thinkpad_data["buckets"]["aw-watcher-window_DESKTOP-MO13KN5"]["events"]
thinkpad_web = thinkpad_data["buckets"]["aw-watcher-web-chrome"]["events"]

df_thinkpad_apps = pd.DataFrame(thinkpad_apps)
df_thinkpad_apps["type"] = "app"
df_thinkpad_web = pd.DataFrame(thinkpad_web)

df = pd.concat([df_thinkpad_web, df_thinkpad_apps])

df["start"] = pd.to_datetime(df["timestamp"], infer_datetime_format=True, utc=True)
df["end"] = df["start"] + pd.to_timedelta(df["duration"], unit="s")
df["duration"] = df["end"] - df["start"]
df = df.sort_values("start")

df["url"] = df["data"].apply(lambda data: data["url"] if "url" in data else None)
df["title"] = df["data"].apply(lambda data: data["title"] if "title" in data else None)
df["app"] = df["data"].apply(lambda data: data["app"] if "app" in data else None)

df = df[["start", "end", "app", "title", "url"]]

df.to_csv("events.csv", index=False)

df